# P1 parellel processing

In [1]:
from pyspark.sql import SQLContext
import pandas as pd
import pymysql
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime, timedelta
import logging
from tqdm import tqdm
from fbprophet import Prophet
from sklearn.metrics import mean_squared_error as mse
import math

## Logging ##

import os
import sys


from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
import os
import pandas as pd

from pyspark.sql import SparkSession
import datetime
# initialise sparkContext
spark1 = SparkSession.builder \
    .master('local') \
    .appName('p7_sample') \
    .config('spark.executor.memory', '24gb') \
    .config("spark.cores.max", "16") \
    .getOrCreate()

sc = spark1.sparkContext

# using SQLContext to read parquet file
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)



In [2]:
def create_prophet_m(app_name,z1):
    
    ### --- For realtime pred ---###
    
    full_df = z1.app_rsp_time.iloc[0:len(z1)]
    full_df = full_df.reset_index()
    full_df.columns = ['ds','y']
    
    #-- Realtime prediction --##
    #model 
    model_r = Prophet(yearly_seasonality=False,changepoint_prior_scale=.2)
    model_r.fit(full_df)
    future_r = model_r.make_future_dataframe(periods=24,freq='H')
    forecast_r = model_r.predict(future_r)
    forecast_r.index = forecast_r['ds']
    #forecast 
    pred_r = pd.DataFrame(forecast_r['yhat'][len(z1):(len(z1)+24)])
    pred_r=pred_r.reset_index()
    #--- completes realtime pred ---#
    
    train_end_index=len(z1.app_rsp_time)-24
    train_df=z1.app_rsp_time.iloc[0:train_end_index]
    #train_df= train_df[train_df<cutter]
    
    
    test_df=z1.app_rsp_time.iloc[train_end_index:len(z1)]
    
    
    
    train_df=train_df.reset_index()
    test_df=test_df.reset_index()
    train_df.columns=['ds','y']
    
    #--- removing outliers in trainset  ---#
    
    q50 = train_df.y.median()
    q100 = train_df.y.quantile(1)
    q75  = train_df.y.quantile(.75)
    #print(max(train_df.y))
    if((q100-q50) >= (2*q50)):
        print('ind')
        train_df.loc[train_df.y>=(2*q50),'y'] = None
    
    test_df.columns=['ds','y']
    
    #model 
    model = Prophet(yearly_seasonality=False,changepoint_prior_scale=.2)
    model.fit(train_df)
    future = model.make_future_dataframe(periods=len(test_df),freq='H')
    forecast = model.predict(future)
    forecast.index = forecast['ds']
    #forecast 
    pred = pd.DataFrame(forecast['yhat'][train_end_index:len(z1)])
    pred=pred.reset_index()
    pred_df=pd.merge(test_df,pred,on='ds',how='left')
    pred_df.dropna(inplace=True)
    
    df=pd.DataFrame()
    
    if(len(pred_df)>0):
        
        pred_df['error_test']=pred_df.y-pred_df.yhat
    
        
    
        MSE=mse(pred_df.y,pred_df.yhat)
        RMSE=math.sqrt(MSE)
        pred_df['APE']=abs(pred_df.error_test*100/pred_df.y)
        MAPE=pred_df.APE.mean()
        print("App name:",app_name)
        print("MSE  :",MSE)
        print("RMSE :",RMSE)
        print("MAPE :",MAPE)
        #print("RMSE from mean :",mean_RMSE)
        #print("MAPE from mean :",mean_MAPE)
    
        #print(pred_df.head())
        q98=pred_df['APE'].quantile(0.98)
        mape_q98=pred_df['APE'][pred_df.APE<pred_df['APE'].quantile(0.98)].mean()

        df = pd.DataFrame({'length':len(z1),#'predicted_t':[forcast_lag],
                             'test_rmse':RMSE,
                             'test_mape':MAPE,
                 #'test_ape_98':q98,
                 'test_mape_98':mape_q98},
                      #'rmse_by_mean':mean_RMSE,
                      #'mape_by_mean':mean_MAPE}
                          index=[app_name])

    return(df,model,forecast,pred_df,pred_r)




In [3]:
df = sqlContext.read.parquet('appid_datapoint_parquet1')

In [4]:
df.head()

Row(time_stamp=1510801200000, source='134.141.5.104', zone='\\N', target_address='10.51.1.95/', application='Sun RPC', applicationgroup='Protocols', target_attribute_id=215904025, byte_count=8776, tx_byte_count=4920, rx_byte_count=3856, flow_count=59, tx_flow_count=29, rx_flow_count=30, tcp_rsp_time=1011, app_rsp_time=3391, user_data='\\N')

In [5]:
import multiprocessing as mp
from joblib import Parallel, delayed
a = 'DNS'
df1 = df[df.application==a]
df1 = df1[df1.app_rsp_time>0]
df1.head()

Row(time_stamp=1510801200000, source='134.141.5.104', zone='\\N', target_address='134.141.124.10/', application='DNS', applicationgroup='Protocols', target_attribute_id=215904026, byte_count=1350, tx_byte_count=599, rx_byte_count=751, flow_count=12, tx_flow_count=6, rx_flow_count=6, tcp_rsp_time=0, app_rsp_time=68705, user_data='\\N')

In [6]:
t1 = datetime.datetime.now()
appid_datapoint_10days = df1
t2 = datetime.datetime.now()
str(t2-t1)
appid_datapoint_10days.head()

Row(time_stamp=1510801200000, source='134.141.5.104', zone='\\N', target_address='134.141.124.10/', application='DNS', applicationgroup='Protocols', target_attribute_id=215904026, byte_count=1350, tx_byte_count=599, rx_byte_count=751, flow_count=12, tx_flow_count=6, rx_flow_count=6, tcp_rsp_time=0, app_rsp_time=68705, user_data='\\N')

In [3]:
app_details = pd.read_csv('app_details.csv')
targets = (app_details.loc[(app_details['count']>=1400),:].target_address.value_counts().index)
s_list = app_details.source.value_counts().index
s = s_list[0]
t = targets[0]

In [8]:
targets2

NameError: name 'targets2' is not defined

In [9]:
df2[df2.target_address.isin(targets2)]

NameError: name 'df2' is not defined

In [10]:
from tqdm import tqdm
temp2 = appid_datapoint_10days[(appid_datapoint_10days.source==s) & (appid_datapoint_10days.target_address==t)]
t1 = datetime.datetime.now()

ts = temp2.select('time_stamp').collect()
ts_array = [int(i.time_stamp) for i in ts]
y = temp2.select('app_rsp_time').collect()
y_array = [int(i.app_rsp_time) for i in y]

t2 = datetime.datetime.now()
print((str(t2-t1)))
dates_outlook = pd.to_datetime(pd.Series(ts_array),unit='ms')
df2 = pd.DataFrame({'ds':ts_array,'app_rsp_time':y_array})
df2.index = dates_outlook  
df2.head()

0:00:32.689130


,app_rsp_time,ds


In [11]:
df1.head()

Row(time_stamp=1510801200000, source='134.141.5.104', zone='\\N', target_address='134.141.124.10/', application='DNS', applicationgroup='Protocols', target_attribute_id=215904026, byte_count=1350, tx_byte_count=599, rx_byte_count=751, flow_count=12, tx_flow_count=6, rx_flow_count=6, tcp_rsp_time=0, app_rsp_time=68705, user_data='\\N')

In [12]:
        t1 = datetime.datetime.now()

        ts = df1.select('time_stamp').collect()
        ts_array = [int(i.time_stamp) for i in ts]
        y = df1.select('app_rsp_time').collect()
        y_array = [int(i.app_rsp_time) for i in y]
        ss = df1.select('source').collect()
        s_array = [str(i.source) for i in ss]
        ta = df1.select('target_address').collect()
        ta_array = [str(i.target_address) for i in ta]
        aa = df1.select('application').collect()
        a_array = [str(i.application) for i in aa]

        
        dates_outlook = pd.to_datetime(pd.Series(ts_array),unit='ms')
        
        df2 = pd.DataFrame({'ds':ts_array,'app_rsp_time':y_array,'application':a_array,
                           'source':s_array,'target_address':ta_array})
        df2.index = dates_outlook
        df2 = df2[df2.app_rsp_time!=0]
        t2 = datetime.datetime.now()
        print((str(t2-t1)))

0:06:59.749956


In [13]:
len(df2)

5622539

In [15]:
#temp = appid_datapoint_10days[appid_datapoint_10days.source==s]

def forapp(t,s,df3):
    
    q1 = datetime.datetime.now()
    #for a in tqdm(a_list[0:1]):
    
    qt1 = datetime.datetime.now()
    
    
    qt2 = datetime.datetime.now()
    query_time = str(qt2-qt1)
    
    # run for all combinations


    prophet_df = pd.DataFrame()
    prophet_analysis_df = pd.DataFrame()
    prophet_future_df = pd.DataFrame()
   
    
    #for i in tqdm(range(0,len(t_list))):
        
    #t = t_list[i]
        #temp1 = temp[temp.target_address==t]
    #for s in tqdm(s_list):
    #if(1):
            
    temp2 = df3[(df3.target_address==t)]
        
        #temp2 = temp[(temp.target_address==t)]
    t1 = datetime.datetime.now()

        #ts = temp2.select('time_stamp').collect()
        #ts_array = [int(i.time_stamp) for i in ts]
        #y = temp2.select('app_rsp_time').collect()
        #y_array = [int(i.app_rsp_time) for i in y]

    #t2 = datetime.datetime.now()
    #print((str(t2-t1)))
        #dates_outlook = pd.to_datetime(pd.Series(ts_array),unit='ms')
        
        #df2 = pd.DataFrame({'ds':ts_array,'app_rsp_time':y_array})
        #df2.index = dates_outlook
        #df2 = df2[df2.app_rsp_time!=0]
        #df2 = df2.dropna()
    if(len(temp2)>1400):
            
        t2 = datetime.datetime.now()
            
        ew_prophet,ew_model,ew_forcast,ew_pred_df,pred_r =(create_prophet_m(a,temp2))
        t2 = datetime.datetime.now()
        print('time for prophet modelling',(str(t2-t1)))
        print('time for prophet modelling',(str(t2-t1)))
        ew_pred_df['application'] = a
        ew_pred_df['target'] = t
        ew_pred_df['source'] = s
            
        pred_r['application'] = a
        pred_r['target'] = t
        pred_r['source'] = s
        
        ew_prophet['target'] = t
        ew_prophet['source'] = s
        ew_prophet['application'] = a
            
        prophet_df = prophet_df.append(ew_pred_df)  
        prophet_analysis_df = prophet_analysis_df.append(ew_prophet)
        prophet_future_df = prophet_future_df.append(pred_r)
        
    qt3 = datetime.datetime.now()
    print('time for all calculation', str(qt3-qt1))
    prophet_df.to_csv("parallel_test/p1_eval_"+s+"_"+t.replace("/","")+"_.csv")
    #prophet_analysis_df.to_csv('parallel_test/p1_analysis_'+s+'_'+t+'.csv')
    #prophet_future_df.to_csv('parallel_test/p1_forecast_'+s+'_'+t+'.csv')
    
    return prophet_df, prophet_analysis_df, prophet_future_df

In [20]:
type(s)

str

In [21]:

t1 = datetime.datetime.now()
t = targets[0]
s = s_list[0]
a1,a2,a3=forapp(t,s,df2[(df2.source==s) & (df2.target_address==t)])
t2 = datetime.datetime.now()
str(t2-t1)

time for all calculation 0:00:00.001604


'0:00:00.910521'

In [49]:
#t1 = datetime.datetime.now()
#targets2 = targets[0:7]
#pool = Parallel(n_jobs=-1,verbose=5,pre_dispatch='all')
#results = pool(delayed(forapp)(t,df2) for t in targets2) #doc_list[0:50000]
#t2 = datetime.datetime.now()
#print((str(t2-t1)))

In [58]:
t4 = datetime.datetime.now()
targets2 = targets[0:16]
with Parallel(n_jobs=-1,verbose=5,pre_dispatch='all') as pool:
    for  s in tqdm(s_list):
        df3 = df2[df2.source==s]
        results = pool(delayed(forapp)(t,s,df3) for t in targets2) #doc_list[0:50000]
t5 = datetime.datetime.now()
print('total time ',(str(t5-t4)))


















  0%|          | 0/7 [00:00<?, ?it/s]

time for all calculation 0:00:00.236769


[Parallel(n_jobs=-1)]: Done   1 out of  16 | elapsed:    8.1s remaining:  2.0min

Exception in thread Thread-51:
Traceback (most recent call last):
  File "/root/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/root/anaconda3/lib/python3.6/site-packages/tqdm/_tqdm.py", line 148, in run
    for instance in self.tqdm_cls._instances:
  File "/root/anaconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



time for all calculation 0:00:00.268729
time for all calculation 0:00:00.202933
time for all calculation 0:00:00.211868
ind
time for all calculation 0:00:00.536098
App name: DNS
MSE  : 164520509.05160066
RMSE : 12826.554839535076
MAPE : 30.14549733640986
time for prophet modelling 0:00:14.848162
time for all calculation 0:00:15.089876


[Parallel(n_jobs=-1)]: Done   5 out of  16 | elapsed:   38.4s remaining:  1.4min


...

time for all calculation 0:00:00.013996
time for all calculation 0:00:00.013455


[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed:    2.7s finished
 86%|████████▌ | 6/7 [03:18<00:33, 33.12s/it]

time for all calculation 0:00:00.002506
time for all calculation 0:00:00.002716
time for all calculation 0:00:00.002369
time for all calculation 0:00:00.002414
time for all calculation 0:00:00.002436
time for all calculation 0:00:00.003004
time for all calculation 0:00:00.002336
time for all calculation 0:00:00.002495
time for all calculation 0:00:00.002938
time for all calculation 0:00:00.002647
time for all calculation 0:00:00.002402
time for all calculation 0:00:00.002476
time for all calculation 0:00:00.002563
time for all calculation 0:00:00.002397
time for all calculation 0:00:00.007690
time for all calculation 0:00:00.002631


[Parallel(n_jobs=-1)]: Done   1 out of  16 | elapsed:    0.0s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done   5 out of  16 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   9 out of  16 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  13 out of  16 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed:    0.1s finished
100%|██████████| 7/7 [03:19<00:00, 28.47s/it]


total time  0:03:28.332266


In [31]:
df2.head()

,app_rsp_time,application,ds,source,target_address
2017-11-16 03:00:00,68705,DNS,1510801200000,134.141.5.104,134.141.124.10/
2017-11-16 03:00:00,639,DNS,1510801200000,134.141.5.104,134.141.90.139/wbl6ycb100
2017-11-16 03:00:00,5345,DNS,1510801200000,134.141.5.104,134.141.22.168/
2017-11-16 03:00:00,105870,DNS,1510801200000,134.141.5.104,134.141.23.9/
2017-11-16 03:00:00,15910,DNS,1510801200000,134.141.5.104,134.141.23.16/


In [32]:
appid_datapoint_10days

DataFrame[time_stamp: bigint, source: string, zone: string, target_address: string, application: string, applicationgroup: string, target_attribute_id: int, byte_count: bigint, tx_byte_count: bigint, rx_byte_count: bigint, flow_count: int, tx_flow_count: int, rx_flow_count: int, tcp_rsp_time: int, app_rsp_time: int, user_data: string]

In [33]:
len(df2)

5622539

In [21]:
df.count()

234800805

In [51]:
results

[(Empty DataFrame
  Columns: []
  Index: [], Empty DataFrame
  Columns: []
  Index: [], Empty DataFrame
  Columns: []
  Index: []), (Empty DataFrame
  Columns: []
  Index: [], Empty DataFrame
  Columns: []
  Index: [], Empty DataFrame
  Columns: []
  Index: []), (Empty DataFrame
  Columns: []
  Index: [], Empty DataFrame
  Columns: []
  Index: [], Empty DataFrame
  Columns: []
  Index: []), (Empty DataFrame
  Columns: []
  Index: [], Empty DataFrame
  Columns: []
  Index: [], Empty DataFrame
  Columns: []
  Index: []), (Empty DataFrame
  Columns: []
  Index: [], Empty DataFrame
  Columns: []
  Index: [], Empty DataFrame
  Columns: []
  Index: []), (Empty DataFrame
  Columns: []
  Index: [], Empty DataFrame
  Columns: []
  Index: [], Empty DataFrame
  Columns: []
  Index: []), (Empty DataFrame
  Columns: []
  Index: [], Empty DataFrame
  Columns: []
  Index: [], Empty DataFrame
  Columns: []
  Index: []), (Empty DataFrame
  Columns: []
  Index: [], Empty DataFrame
  Columns: []
  Index:

In [54]:
dummy = pd.read_csv('parallel_test/p1_eval_10.6.1.101_.csv')
dummy.source.unique()

array(['10.6.1.101'], dtype=object)

In [36]:
(results[0][2])

""


## For dynamic calculation

In [20]:
from pyspark.sql import SQLContext
import pandas as pd
import pymysql
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime, timedelta
import logging
from tqdm import tqdm
from fbprophet import Prophet
from sklearn.metrics import mean_squared_error as mse
import math

## Logging ##

import os
import sys


from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
import os
import pandas as pd

from pyspark.sql import SparkSession
import datetime
full_t1 = datetime.datetime.now()
# initialise sparkContext
spark1 = SparkSession.builder \
    .master('local') \
    .appName('p7_sample') \
    .config('spark.executor.memory', '24gb') \
    .config("spark.cores.max", "16") \
    .getOrCreate()

sc = spark1.sparkContext

# using SQLContext to read parquet file
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

def create_prophet_m(app_name,z1,delay=24):
    
    ### --- For realtime pred ---###
    
    full_df = z1.app_rsp_time.iloc[0:len(z1)]
    full_df = full_df.reset_index()
    full_df.columns = ['ds','y']
    
    #-- Realtime prediction --##
    #model 
    model_r = Prophet(yearly_seasonality=False,changepoint_prior_scale=.2)
    model_r.fit(full_df)
    future_r = model_r.make_future_dataframe(periods=delay,freq='H')
    forecast_r = model_r.predict(future_r)
    forecast_r.index = forecast_r['ds']
    #forecast 
    pred_r = pd.DataFrame(forecast_r['yhat'][len(z1):(len(z1)+delay)])
    pred_r=pred_r.reset_index()
    #--- completes realtime pred ---#
    
    train_end_index=len(z1.app_rsp_time)-delay
    train_df=z1.app_rsp_time.iloc[0:train_end_index]
    #train_df= train_df[train_df<cutter]
    
    
    test_df=z1.app_rsp_time.iloc[train_end_index:len(z1)]
    
    
    
    train_df=train_df.reset_index()
    test_df=test_df.reset_index()
    train_df.columns=['ds','y']
    
    #--- removing outliers in trainset  ---#
    
    q50 = train_df.y.median()
    q100 = train_df.y.quantile(1)
    q75  = train_df.y.quantile(.75)
    #print(max(train_df.y))
    if((q100-q50) >= (2*q50)):
        print('ind')
        train_df.loc[train_df.y>=(2*q50),'y'] = None
    
    test_df.columns=['ds','y']
    print('len of testdf = ',len(test_df))
    #model 
    model = Prophet(yearly_seasonality=False,changepoint_prior_scale=.2)
    model.fit(train_df)
    future = model.make_future_dataframe(periods=len(test_df),freq='H')
    forecast = model.predict(future)
    forecast.index = forecast['ds']
    #forecast 
    pred = pd.DataFrame(forecast['yhat'][train_end_index:len(z1)])
    pred=pred.reset_index()
    pred_df=pd.merge(test_df,pred,on='ds',how='left')
    pred_df.dropna(inplace=True)
    
    df=pd.DataFrame()
    
    if(len(pred_df)>0):
        
        pred_df['error_test']=pred_df.y-pred_df.yhat
    
        
    
        MSE=mse(pred_df.y,pred_df.yhat)
        RMSE=math.sqrt(MSE)
        pred_df['APE']=abs(pred_df.error_test*100/pred_df.y)
        MAPE=pred_df.APE.mean()
        print("App name:",app_name)
        print("MSE  :",MSE)
        print("RMSE :",RMSE)
        print("MAPE :",MAPE)
        
        q98=pred_df['APE'].quantile(0.98)
        mape_q98=pred_df['APE'][pred_df.APE<pred_df['APE'].quantile(0.98)].mean()

        df = pd.DataFrame({'length':len(z1),#'predicted_t':[forcast_lag],
                             'test_rmse':RMSE,
                             'test_mape':MAPE,
                 #'test_ape_98':q98,
                 'test_mape_98':mape_q98},
                   
                          index=[app_name])

    return(df,model,forecast,pred_df,pred_r)



In [23]:
df = sqlContext.read.parquet('appid_datapoint_parquet1')
df = df[df.app_rsp_time!=0]
def forapp(t,s,df3):
    
    q1 = datetime.datetime.now()
    #for a in tqdm(a_list[0:1]):
    
    # run for all combinations


    prophet_df = pd.DataFrame()
    prophet_analysis_df = pd.DataFrame()
    prophet_future_df = pd.DataFrame()
   
    temp2 = df3
    
    t1 = datetime.datetime.now()

    if(len(temp2)>1400):
            
        t2 = datetime.datetime.now()
            
        ew_prophet,ew_model,ew_forcast,ew_pred_df,pred_r =(create_prophet_m(a,temp2,24))
        t2 = datetime.datetime.now()
        print('time for prophet modelling',(str(t2-t1)))
        print('time for prophet modelling',(str(t2-t1)))
        ew_pred_df['application'] = a
        ew_pred_df['target'] = t
        ew_pred_df['source'] = s
            
        pred_r['application'] = a
        pred_r['target'] = t
        pred_r['source'] = s
        
        ew_prophet['target'] = t
        ew_prophet['source'] = s
        ew_prophet['application'] = a
            
        prophet_df = prophet_df.append(ew_pred_df)  
        prophet_analysis_df = prophet_analysis_df.append(ew_prophet)
        prophet_future_df = prophet_future_df.append(pred_r)
        
    qt3 = datetime.datetime.now()
    #print('time for all calculation', str(qt3-qt1))
    prophet_df.to_csv("parallel_test/p1_eval_"+s+"_"+t.replace("/","")+"_.csv")
    #prophet_analysis_df.to_csv('parallel_test/p1_analysis_'+s+'_'+t+'.csv')
    #prophet_future_df.to_csv('parallel_test/p1_forecast_'+s+'_'+t+'.csv')
    
    return prophet_df, prophet_analysis_df, prophet_future_df

t1 = datetime.datetime.now()
#t = '10.6.16.32/' #targets[0]
#s = s_list[1]
s = '10.6.1.101'
a='DNS'


#a1,a2,a3=forapp(t,s,df2[(df2.source==s) & (df2.target_address==t)])
A1=pd.DataFrame()
A2=pd.DataFrame()
A3=pd.DataFrame()
for i in range(0,10):
    t = ta_array[i]
    
    t1 = datetime.datetime.now()
    df1 = df[(df.target_address == t) & (df.application == a) & (df.source == s)]

    #ts = df1.select('time_stamp').collect()
    #ts_array = [int(i.time_stamp) for i in ts]
    #y = df1.select('app_rsp_time').collect()
    #y_array = [int(i.app_rsp_time) for i in y]
    #ss = df1.select('source').collect()
    #s_array = [str(i.source) for i in ss]
    #ta = df1.select('target_address').collect()
    #ta_array = [str(i.target_address) for i in ta]
    #aa = df1.select('application').collect()
    #a_array = [str(i.application) for i in aa]
        
    #dates_outlook = pd.to_datetime(pd.Series(ts_array),unit='ms')
        
    #df2 = pd.DataFrame({'ds':ts_array,'app_rsp_time':y_array })#,'application':a_array,
                           #'source':s_array,'target_address':ta_array})
    df2 = df1.toPandas()
    dates_outlook = pd.to_datetime(pd.Series(df2.time_stamp.astype(int)),unit='ms')
    df2.index = dates_outlook
    df2 = df2[df2.app_rsp_time!=0]
    df2 = df2.sort_values(by='time_stamp')
    t2 = datetime.datetime.now()
    print((str(t2-t1)))
    
    a1,a2,a3=forapp(t,s,df2)
    A1 = A1.append(a1)
    A2 = A2.append(a2)
    A3 = A3.append(a3)
t2 = datetime.datetime.now()
full_t2 = datetime.datetime.now()
str(full_t2-full_t1)

0:00:21.577868
len of testdf =  24
App name: DNS
MSE  : 605968577.3680652
RMSE : 24616.429013324927
MAPE : 11.534673914457153
time for prophet modelling 0:00:11.330581
time for prophet modelling 0:00:11.330581
0:00:21.460914
len of testdf =  24
App name: DNS
MSE  : 1207885355.7872488
RMSE : 34754.64509655147
MAPE : 7.853658218903047
time for prophet modelling 0:00:12.705812
time for prophet modelling 0:00:12.705812
0:00:21.582631
len of testdf =  24
App name: DNS
MSE  : 15350942953.538794
RMSE : 123898.92232597826
MAPE : 2.492707507414026
time for prophet modelling 0:00:10.726708
time for prophet modelling 0:00:10.726708
0:00:21.546631
ind
len of testdf =  24
App name: DNS
MSE  : 901752848.7171904
RMSE : 30029.199934683413
MAPE : 24.568138214464483
time for prophet modelling 0:00:10.356018
time for prophet modelling 0:00:10.356018
0:00:21.339514
len of testdf =  24
App name: DNS
MSE  : 3432061085.9154124
RMSE : 58583.795420879076
MAPE : 15.040133401783349
time for prophet modelling 0:0

'0:07:06.320055'

In [16]:
gg = df1.toPandas()

In [17]:
gg.head()

,time_stamp,source,zone,target_address,application,applicationgroup,target_attribute_id,byte_count,tx_byte_count,rx_byte_count,flow_count,tx_flow_count,rx_flow_count,tcp_rsp_time,app_rsp_time,user_data
0,1511780400000,10.6.1.101,\N,10.6.16.102/,DNS,Protocols,218165936,335390,205490,129900,2311,1155,1156,0,1377,\N
1,1511784000000,10.6.1.101,\N,10.6.16.102/,DNS,Protocols,218177710,300937,177618,123319,2032,1013,1019,423,1067,\N
2,1511787600000,10.6.1.101,\N,10.6.16.102/,DNS,Protocols,218191976,342148,201218,140930,2302,1149,1153,3830,1042,\N
3,1511791200000,10.6.1.101,\N,10.6.16.102/,DNS,Protocols,218204485,273854,164760,109094,1978,986,992,34422,1344,\N
4,1511794800000,10.6.1.101,\N,10.6.16.102/,DNS,Protocols,218219419,168052,98541,69511,1313,654,659,32585,1664,\N


In [ ]:
ta = df[(df.source==s) & (df.application==a)].select('target_address').collect()
ta_array = [str(i.target_address) for i in ta]
ta_array

In [38]:
a1

,ds,y,yhat,error_test,APE,application,target,source
161,2018-02-09 12:00:00,188752,194837.320412,-6085.320412,3.223977,DNS,10.6.16.32/,10.6.1.101
162,2018-02-09 13:00:00,174230,182711.713000,-8481.713000,4.868113,DNS,10.6.16.32/,10.6.1.101
163,2018-02-09 14:00:00,142512,171699.396027,-29187.396027,20.480658,DNS,10.6.16.32/,10.6.1.101
164,2018-02-09 15:00:00,141531,161486.218561,-19955.218561,14.099539,DNS,10.6.16.32/,10.6.1.101
165,2018-02-09 16:00:00,147162,153916.583695,-6754.583695,4.589897,DNS,10.6.16.32/,10.6.1.101
166,2018-02-09 17:00:00,138544,151893.234973,-13349.234973,9.635376,DNS,10.6.16.32/,10.6.1.101
167,2018-02-09 18:00:00,123481,155219.754597,-31738.754597,25.703351,DNS,10.6.16.32/,10.6.1.101


In [21]:
df[(df.source==s) & (df.application == a)].head(3)

[Row(time_stamp=1511780400000, source='10.6.1.101', zone='\\N', target_address='10.6.16.32/', application='DNS', applicationgroup='Protocols', target_attribute_id=218165899, byte_count=73951948, tx_byte_count=46738951, rx_byte_count=27212997, flow_count=492880, tx_flow_count=246326, rx_flow_count=246554, tcp_rsp_time=224, app_rsp_time=244753, user_data='\\N'),
 Row(time_stamp=1511780400000, source='10.6.1.101', zone='\\N', target_address='10.6.25.30/', application='DNS', applicationgroup='Protocols', target_attribute_id=218165900, byte_count=42292759, tx_byte_count=17802393, rx_byte_count=24490366, flow_count=235158, tx_flow_count=118308, rx_flow_count=116850, tcp_rsp_time=131, app_rsp_time=304750, user_data='\\N'),
 Row(time_stamp=1511780400000, source='10.6.1.101', zone='\\N', target_address='134.141.79.201/usnh-cdc-p1', application='DNS', applicationgroup='Protocols', target_attribute_id=218165906, byte_count=16225133, tx_byte_count=7144320, rx_byte_count=9080813, flow_count=108399,